<a href="https://colab.research.google.com/github/ArthisreeRamesh/GenAI-Agentic-AI/blob/main/Aug3EndTevvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install langchain-chroma
!pip install ragas
!pip install deepeval
!pip install dill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 32.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing ins

In [ ]:
from getpass import getpass
OPEN_API_KEY=getpass("Enter api key")

Enter api key··········


In [ ]:
import os
os.environ['OPENAI_API_KEY'] = OPEN_API_KEY

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddingmodel= OpenAIEmbeddings(model="text-embedding-3-small")


In [ ]:
#this is to download the data but we have it handy so directly uploading
#!gdown 1QkSY9W5RyaBnY8c5FLIsmpPVXoHTQ-fb

In [ ]:
import pandas as pd

In [ ]:
df= pd.read_csv("/content/sample_data/rag_eval_docs.csv")
df

,id,title,context
0,1,Machine Learning,Machine learning is a field of artificial inte...
1,2,Deep Learning,Deep learning is a subset of machine learning ...
2,3,Natural Language Processing (NLP),NLP is a branch of AI that enables computers t...
3,4,Pyramids,"Pyramids are ancient structures, often serving..."
4,5,Photosynthesis,Photosynthesis is the process plants use to co...
5,6,Biology,"Biology is the study of living organisms, cove..."
6,7,Quantum Mechanics,Quantum mechanics is a branch of physics that ...
7,8,Cryptocurrency,Cryptocurrency is a digital currency that uses...
8,9,Renewable Energy,"Renewable energy sources, such as solar and wi..."
9,10,Artificial Intelligence,Artificial intelligence refers to machines mim...


orient	   Structure               produced	Example output
'records'	List of row dicts	       [{'col1': val1, 'col2': val2}, ...]
'dict'	  Dict of columns	         {'col1': [..], 'col2': [..]}
'list'	  Dict of columns as lists {'col1': [...], 'col2': [...]}
'index'	  Dict of index to dicts	 {0: {'col1': val1, ...}, 1: {...}}

The orient parameter controls how the DataFrame is converted to a dictionary.

In [ ]:

docs = df.to_dict(orient='records')
docs

[{'id': 1,
  'title': 'Machine Learning',
  'context': 'Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition.'},
 {'id': 2,
  'title': 'Deep Learning',
  'context': 'Deep learning is a subset of machine learning utilizing neural networks with many layers. It excels in complex tasks like image and speech recognition. Convolutional and recurrent neural networks are among the common architectures used.'},
 {'id': 3,
  'title': 'Natural Language Processing (NLP)',
  'context': 'NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.'},
 {'id': 4,
  'title': 'Pyramids',
  'context': 'Pyramids are ancient structur

In [ ]:
#convert to metadata(which is Langchain format to proceed with further processing)
from langchain.docstore.document import Document
precessed_docs = []

for doc in docs:
  metadata = {
      "title": doc['title'],
      "id": doc['id'],
  }
  data = doc['context']
  precessed_docs.append(Document(page_content=data, metadata=metadata))

this whol process of csv -> dict -> metadata could be done by Langchain csv loader directly

In [ ]:
precessed_docs

[Document(metadata={'title': 'Machine Learning', 'id': 1}, page_content='Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition.'),
 Document(metadata={'title': 'Deep Learning', 'id': 2}, page_content='Deep learning is a subset of machine learning utilizing neural networks with many layers. It excels in complex tasks like image and speech recognition. Convolutional and recurrent neural networks are among the common architectures used.'),
 Document(metadata={'title': 'Natural Language Processing (NLP)', 'id': 3}, page_content='NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.'),
 Document(metadata={'title

Index Document and Embedding in vector db

In [ ]:
from langchain_chroma import Chroma
chroma_db= Chroma.from_documents(documents=precessed_docs,embedding =embeddingmodel,collection_name='my_eval',collection_metadata={"hnsw:space":'cosine'},
                                 persist_directory='/myevaldb')

In [ ]:
#load from disc
chroma_db =  Chroma(embedding_function =embeddingmodel,collection_name='my_eval',
                                 persist_directory='/myevaldb')

In [ ]:
chroma_db

#semaintic similarity based retrieval

In [ ]:
#score_threshold- if similarity score is less than score_threshold it wont be considered as relavant document, on top of this top N documents will be filtered based on k=N

similarity_retriever= chroma_db.as_retriever(search_type="similarity_score_threshold",search_kwargs={ "score_threshold" : 0.3 , "k" : 3}
                                             )

In [ ]:
#just to view the retrieved documents in a good format
from IPython.display import display, Markdown

def display_docs(docs):
  for doc in docs:
    print("Metadata :", doc.metadata)
    print("Content Brief:")
    display(Markdown(doc.page_content))
    print()

In [ ]:
query='what is AI'
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

Metadata : {'id': 10, 'title': 'Artificial Intelligence'}
Content Brief:


Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.


Metadata : {'title': 'Natural Language Processing (NLP)', 'id': 3}
Content Brief:


NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.


Metadata : {'id': 1, 'title': 'Machine Learning'}
Content Brief:


Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition.

here only one doucment is retrieved though k=3 because only 1 doc in the database is related to photosynthesis

In [ ]:
query='what is Photosynthesis'
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

Metadata : {'id': 5, 'title': 'Photosynthesis'}
Content Brief:


Photosynthesis is the process plants use to convert sunlight into energy. This process produces glucose and releases oxygen as a byproduct. It is crucial for sustaining life on Earth by providing food and oxygen.

RAG pipeline

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
rag_prompt = """You are an assistant who is an expert in question-answering tasks.
              Answer the following question using only the following pieces of retrieved context.
              If the answer is not in the context, do not make up answers, just say that you don't know.
              Keep the answer to the points based on the information from the context.

              Question: {question}
              Context: {context}

              Answer:


              """
rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

similarity_retriever (Team 1) → Fetches relevant docs.

src_rag_response_chain (Team 2) → Formats docs + prompts GPT + parses output.

Each chain does its own job well.
If you merged them into one huge function, debugging, replacing parts, or improving performance (like caching retrieval results) becomes messy.

So having two stages = modularity + maintainability.

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

src_rag_response_chain = ({
    "context":(itemgetter('context')
    | RunnableLambda(format_docs)),
               "question":itemgetter("question")}
               | rag_prompt_template
               | chatgpt
               | StrOutputParser() )

rag_chain_w_sources = (
    {"context": similarity_retriever, "question": RunnablePassthrough()}
    | RunnablePassthrough.assign(
        response=src_rag_response_chain,
    )
)




In [ ]:
query = "how do plants survive?"
result = rag_chain_w_sources.invoke(query)
result

{'context': [Document(id='4783b169-b905-4d2b-b529-97ab6b54e897', metadata={'id': 5, 'title': 'Photosynthesis'}, page_content='Photosynthesis is the process plants use to convert sunlight into energy. This process produces glucose and releases oxygen as a byproduct. It is crucial for sustaining life on Earth by providing food and oxygen.')],
 'question': 'how do plants survive?',
 'response': 'Plants survive by using a process called photosynthesis to convert sunlight into energy. This process produces glucose, which serves as food for the plant, and releases oxygen as a byproduct.'}

In [ ]:
query = "What is Artificial Intelligence?"
result = rag_chain_w_sources.invoke(query)
result

{'context': [Document(id='4be2dad3-b269-49c0-9193-906e65d704da', metadata={'title': 'Artificial Intelligence', 'id': 10}, page_content="Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning."),
  Document(id='155edbf1-acda-40fc-993a-eceb437cd68a', metadata={'title': 'Machine Learning', 'id': 1}, page_content='Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition.'),
  Document(id='a3a26008-cbe0-45a9-897b-962d39d3fb54', metadata={'title': 'Natural Language Processing (NLP)', 'id': 3}, page_content='NLP is a branch of AI that enables computers to understand, interpret, and

evaluation


In [ ]:
retrieved_context = [doc.page_content for doc in result['context']]
retrieved_context

["Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.",
 'Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition.',
 'NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.']

In [ ]:
human_answer = """AI, also known as Artificial Intelligence is used to build complex systems for applications
                like virtual assistants, robotics and autonomous vehicles."""

In [ ]:
#just for us to understand experiment purpose
new_context =["Machine Learning is the study of algorithm which learn with more data",
              "AI is known as Artificial Intelligence "] + retrieved_context

evaluation metrics

In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualPrecisionMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input = result['question'],
    actual_output=result['response'],
    expected_output=human_answer,
    retrieval_context=retrieved_context
)

metrics = ContextualPrecisionMetric(
    threshold=0.55,
    model ='gpt-4o',
    include_reason=True,
    verbose_mode=True)

result = evaluate([test_case], [metrics])

TypeError: 'EvaluationResult' object is not subscriptable

In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualPrecisionMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input = result['question'],
    actual_output=result['response'],
    expected_output=human_answer,
    retrieval_context=new_context
)

metrics = ContextualPrecisionMetric(
    threshold=0.55,
    model ='gpt-4o',
    include_reason=True,
    verbose_mode=True)

result = evaluate([test_case], [metrics])

✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases


**************************************************

Contextual Precision Verbose Logs

...

Verdicts:
[
    {
        "verdict": "no",
        "reason": "The context 'Machine Learning is the study of algorithm which learn with more data' does not 
directly address the question about Artificial Intelligence or its applications."
    },
    {
        "verdict": "yes",
        "reason": "The context 'AI is known as Artificial Intelligence' directly defines AI, which is relevant to 
the input question."
    },
    {
        "verdict": "yes",
        "reason": "The context 'Artificial intelligence refers to machines mimicking human intelligence, like 
problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles.'
directly addresses the question and matches the expected output."
    },
    {
        "verdict": "no",
        "reason": "The context 'Machine learning is a field of artificial intelligence focused on enabling systems 
to learn patterns from data.' focuses on machine learning rather than AI as a whole, and does not mention 
applications like virtual assistants or autonomous vehicles."
    },
    {
        "verdict": "no",
        "reason": "The context 'NLP is a branch of AI that enables computers to understand, interpret, and generate
human language.' focuses on NLP, a specific branch of AI, and does not address the broader question about AI and 
its applications."
    }
]
 
Score: 0.5833333333333333
Reason: The score is 0.58 because the first node, ranked highest, contains the context 'Machine Learning is the 
study of algorithm which learn with more data', which does not directly address the question about Artificial 
Intelligence or its applications. This irrelevant node should be ranked lower. However, the second and third nodes 
provide relevant information, with the second node defining AI and the third node explaining its applications, 
which justifies the current score.

======================================================================



Metrics Summary

  - ✅ Contextual Precision (score: 0.5833333333333333, threshold: 0.55, strict: False, evaluation model: gpt-4o, reason: The score is 0.58 because the first node, ranked highest, contains the context 'Machine Learning is the study of algorithm which learn with more data', which does not directly address the question about Artificial Intelligence or its applications. This irrelevant node should be ranked lower. However, the second and third nodes provide relevant information, with the second node defining AI and the third node explaining its applications, which justifies the current score., error: None)

For test case:

  - input: What is Artificial Intelligence?
  - actual output: Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.
  - expected output

⚠ WARNING: No hyperparameters logged.
» ]8;id=786466;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 7.54s | token cost: 0.006502500000000001 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [ ]:
print("Success:", result.test_results[0].metrics_data[0].success)
print("Score:", result.test_results[0].metrics_data[0].score)
print("Reason:", result.test_results[0].metrics_data[0].reason)

Success: True
Score: 0.5833333333333333
Reason: The score is 0.58 because the first node, ranked highest, contains the context 'Machine Learning is the study of algorithm which learn with more data', which does not directly address the question about Artificial Intelligence or its applications. This irrelevant node should be ranked lower. However, the second and third nodes provide relevant information, with the second node defining AI and the third node explaining its applications, which justifies the current score.


In [ ]:
new_context1 =["NVIDIA makes chips for AI",
              "AI is known as Artificial Intelligence ",
              "Physics and Maths are simple",
              "Chemistry is really simple if you know the fundamentals"] + retrieved_context

In [ ]:
human_answer = """AI, also known as Artificial Intelligence is used to build complex systems for applications
                like virtual assistants, robotics and autonomous vehicles."""

In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualRecallMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input = result['question'],
    actual_output=result['response'],
    expected_output=human_answer,
    retrieval_context=new_context1
)

metrics = ContextualRecallMetric(
    threshold=0.55,
    model ='gpt-4o',
    include_reason=True,
    verbose_mode=True)

result = evaluate([test_case], [metrics])

✨ You're running DeepEval's latest Contextual Recall Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases


**************************************************

Contextual Recall Verbose Logs

...

Verdicts:
[
    {
        "verdict": "yes",
        "reason": "The sentence can be attributed to the 2nd and 5th nodes. The 2nd node states 'AI is known as 
Artificial Intelligence', and the 5th node mentions 'AI includes applications like virtual assistants, robotics, 
and autonomous vehicles...'."
    }
]
 
Score: 1.0
Reason: The score is 1.00 because every sentence in the expected output is perfectly aligned with the nodes in the 
retrieval context, specifically the 2nd and 5th nodes, which provide comprehensive support for the information 
presented.

======================================================================



Metrics Summary

  - ✅ Contextual Recall (score: 1.0, threshold: 0.55, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because every sentence in the expected output is perfectly aligned with the nodes in the retrieval context, specifically the 2nd and 5th nodes, which provide comprehensive support for the information presented., error: None)

For test case:

  - input: What is Artificial Intelligence?
  - actual output: Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.
  - expected output: AI, also known as Artificial Intelligence is used to build complex systems for applications
                like virtual assistants, robotics and autonomous vehicles.
  - context: None
  - retrieval context: ['NVIDIA makes chips for AI', 'AI is known as Artifici

⚠ WARNING: No hyperparameters logged.
» ]8;id=31719;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 3.88s | token cost: 0.0037450000000000005 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

in above case (recall) we are getting 100 percent accuracy even though there are bit non relevant documents because false negative is 0. and formula is TP/(FN+TP)

In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualRelevancyMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input = result['question'],
    actual_output=result['response'],
    expected_output=human_answer,
    retrieval_context=new_context
)

metrics = ContextualRelevancyMetric(
    threshold=0.55,
    model ='gpt-4o',
    include_reason=True,
    verbose_mode=True)

result = evaluate([test_case], [metrics])

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases


**************************************************

Contextual Relevancy Verbose Logs

...

Verdicts:
[
    {
        "verdicts": [
            {
                "statement": "NVIDIA makes chips for AI",
                "verdict": "yes",
                "reason": null
            }
        ]
    },
    {
        "verdicts": [
            {
                "statement": "AI is known as Artificial Intelligence",
                "verdict": "yes",
                "reason": null
            }
        ]
    },
    {
        "verdicts": [
            {
                "statement": "Physics and Maths are simple",
                "verdict": "no",
                "reason": "The statement 'Physics and Maths are simple' does not relate to the input question 'What
is Artificial Intelligence?' as it discusses unrelated subjects."
            }
        ]
    },
    {
        "verdicts": [
            {
                "statement": "Chemistry is really simple if you know the fundamentals",
                "verdict": "no",
                "reason": "The statement 'Chemistry is really simple if you know the fundamentals' is unrelated to 
the topic of Artificial Intelligence."
            }
        ]
    },
    {
        "verdicts": [
            {
                "statement": "Artificial intelligence refers to machines mimicking human intelligence, like 
problem-solving and learning.",
                "verdict": "yes",
                "reason": null
            },
            {
                "statement": "AI includes applications like virtual assistants, robotics, and autonomous 
vehicles.",
                "verdict": "yes",
                "reason": null
            },
            {
                "statement": "It's evolving rapidly with advancements in machine learning and deep learning.",
                "verdict": "yes",
                "reason": null
            }
        ]
    },
    {
        "verdicts": [
            {
                "statement": "Machine learning is a field of artificial intelligence focused on enabling systems to
learn patterns from data.",
                "verdict": "yes",
                "reason": null
            },
            {
                "statement": "Algorithms analyze past data to make predictions or classify information.",
                "verdict": "yes",
                "reason": null
            },
            {
                "statement": "Popular applications include recommendation systems and image recognition.",
                "verdict": "yes",
                "reason": null
            }
        ]
    },
    {
        "verdicts": [
            {
                "statement": "NLP is a branch of AI that enables computers to understand, interpret, and generate 
human language.",
                "verdict": "yes",
                "reason": null
            },
            {
                "statement": "Techniques include tokenization, stemming, and sentiment analysis.",
                "verdict": "yes",
                "reason": null
            },
            {
                "statement": "Applications range from chatbots to language translation services.",
                "verdict": "yes",
                "reason": null
            }
        ]
    }
]
 
Score: 0.8461538461538461
Reason: The score is 0.85 because while the retrieval context contains several relevant statements about AI, such 
as 'Artificial intelligence refers to machines mimicking human intelligence' and 'AI includes applications like 
virtual assistants, robotics, and autonomous vehicles,' it also includes unrelated content like 'Physics and Maths 
are simple,' which detracts from the overall relevancy.

======================================================================



Metrics Summary

  - ✅ Contextual Relevancy (score: 0.8461538461538461, threshold: 0.55, strict: False, evaluation model: gpt-4o, reason: The score is 0.85 because while the retrieval context contains several relevant statements about AI, such as 'Artificial intelligence refers to machines mimicking human intelligence' and 'AI includes applications like virtual assistants, robotics, and autonomous vehicles,' it also includes unrelated content like 'Physics and Maths are simple,' which detracts from the overall relevancy., error: None)

For test case:

  - input: What is Artificial Intelligence?
  - actual output: Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.
  - expected output: AI, also known as Artificial Intelligence is used to build complex systems for appli

⚠ WARNING: No hyperparameters logged.
» ]8;id=944448;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 4.17s | token cost: 0.015230000000000002 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

Generator metrics
since its the chcekpoint in generation part of rag pipeline in test case no need to give context similar to retrieval metrics

In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input = result['question'],
    actual_output=result['response'],
)

metrics = AnswerRelevancyMetric(
    threshold=0.55,
    model ='gpt-4o',
    include_reason=True,
    verbose_mode=True)

result = evaluate([test_case], [metrics])

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases


**************************************************

Answer Relevancy Verbose Logs

...

Statements:
[
    "Artificial intelligence refers to machines mimicking human intelligence.",
    "AI includes applications like virtual assistants, robotics, and autonomous vehicles.",
    "AI is evolving rapidly with advancements in machine learning and deep learning."
] 
 
Verdicts:
[
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    }
]
 
Score: 1.0
Reason: The score is 1.00 because the response perfectly addresses the question about Artificial Intelligence 
without any irrelevant information. Great job!

======================================================================



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.55, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the response perfectly addresses the question about Artificial Intelligence without any irrelevant information. Great job!, error: None)

For test case:

  - input: What is Artificial Intelligence?
  - actual output: Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.
  - expected output: None
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

Answer Relevancy: 100.00% pass rate




⚠ WARNING: No hyperparameters logged.
» ]8;id=91555;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 4.21s | token cost: 0.0039025 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import FaithfulnessMetric, HallucinationMetric, GEval
from deepeval import evaluate

test_case = LLMTestCase(
    input = result['question'],
    actual_output=result['response'],
    retrieval_context=new_context
)

metrics = FaithfulnessMetric(
    threshold=0.55,
    model ='gpt-4o',
    include_reason=True,
    verbose_mode=True)

result = evaluate([test_case], [metrics])

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases


**************************************************

Faithfulness Verbose Logs

...

Truths (limit=None):
[
    "Machine Learning is the study of algorithms that learn with more data.",
    "AI is known as Artificial Intelligence.",
    "Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning.",
    "AI includes applications like virtual assistants, robotics, and autonomous vehicles.",
    "AI is evolving rapidly with advancements in machine learning and deep learning.",
    "Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from 
data.",
    "Machine learning algorithms analyze past data to make predictions or classify information.",
    "Popular applications of machine learning include recommendation systems and image recognition.",
    "NLP is a branch of AI that enables computers to understand, interpret, and generate human language.",
    "NLP techniques include tokenization, stemming, and sentiment analysis.",
    "NLP applications range from chatbots to language translation services."
] 
 
Claims:
[
    "Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning.",
    "AI includes applications like virtual assistants, robotics, and autonomous vehicles.",
    "AI is evolving rapidly with advancements in machine learning and deep learning."
] 
 
Verdicts:
[
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    },
    {
        "verdict": "yes",
        "reason": null
    }
]
 
Score: 1.0
Reason: The score is 1.00 because there are no contradictions between the actual output and the retrieval context. 
Everything aligns perfectly, showcasing a high level of accuracy and consistency. Keep up the great work!

======================================================================



Metrics Summary

  - ✅ Faithfulness (score: 1.0, threshold: 0.55, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because there are no contradictions between the actual output and the retrieval context. Everything aligns perfectly, showcasing a high level of accuracy and consistency. Keep up the great work!, error: None)

For test case:

  - input: What is Artificial Intelligence?
  - actual output: Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.
  - expected output: None
  - context: None
  - retrieval context: ['Machine Learning is the study of algorithm which learn with more data', 'AI is known as Artificial Intelligence ', "Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes

⚠ WARNING: No hyperparameters logged.
» ]8;id=303794;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 6.03s | token cost: 0.0073075000000000015 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import FaithfulnessMetric, HallucinationMetric, GEval
from deepeval import evaluate

test_case = LLMTestCase(
    input = result['question'],
    actual_output=result['response'],
    context=new_context1
)

metrics = HallucinationMetric(
    threshold=0.55,
    model ='gpt-4o',
    include_reason=True,
    verbose_mode=True)

result = evaluate([test_case], [metrics])

✨ You're running DeepEval's latest Hallucination Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases


**************************************************

Hallucination Verbose Logs

...

Verdicts:
[
    {
        "verdict": "no",
        "reason": "The actual output does not mention NVIDIA or its role in making chips for AI."
    },
    {
        "verdict": "no",
        "reason": "The actual output does not explicitly define AI as Artificial Intelligence."
    },
    {
        "verdict": "no",
        "reason": "The actual output does not address the simplicity of Physics and Maths."
    },
    {
        "verdict": "no",
        "reason": "The actual output does not discuss the simplicity of Chemistry."
    },
    {
        "verdict": "yes",
        "reason": "The actual output agrees with the context, describing AI as machines mimicking human 
intelligence and listing similar applications."
    },
    {
        "verdict": "no",
        "reason": "The actual output does not specifically focus on machine learning or its applications like 
recommendation systems and image recognition."
    },
    {
        "verdict": "no",
        "reason": "The actual output does not mention NLP or its techniques and applications."
    }
]
 
Score: 0.8571428571428571
Reason: The score is 0.86 because the actual output aligns with the context in describing AI broadly but fails to 
address several specific details and topics mentioned in the context, such as NVIDIA's role, explicit definitions, 
and specific AI applications like machine learning and NLP.

======================================================================



Metrics Summary

  - ❌ Hallucination (score: 0.8571428571428571, threshold: 0.55, strict: False, evaluation model: gpt-4o, reason: The score is 0.86 because the actual output aligns with the context in describing AI broadly but fails to address several specific details and topics mentioned in the context, such as NVIDIA's role, explicit definitions, and specific AI applications like machine learning and NLP., error: None)

For test case:

  - input: What is Artificial Intelligence?
  - actual output: Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.
  - expected output: None
  - context: ['NVIDIA makes chips for AI', 'AI is known as Artificial Intelligence ', 'Physics and Maths are simple', 'Chemistry is really simple if you know the fundamentals', "Artificial intel

⚠ WARNING: No hyperparameters logged.
» ]8;id=527222;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 6.75s | token cost: 0.0047550000000000005 USD)
» Test Results (1 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

no verbose mode/no reason paramter allowed in Geval

Geval customized evaluation metrix (checks quality)

It mainly depends on these 5 criterias:

Relevance -	Alignment with user question	(Semantic similarity, cosine distance, attention alignment)
Coherence	-Logical flow between sentences	(Discourse modeling, attention dependency, perplexity)
Conciseness	-Minimal but sufficient length	(Information density, MDL principle)
Fluency -	Grammatical and natural phrasing	(LM probability, syntax modeling)
Helpfulness	-Practical usefulness of response	(RLHF reward models, utility optimization)


In [ ]:
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import GEval
from deepeval import evaluate

# Define your test case
test_case = LLMTestCase(
    input=result['question'],
    actual_output=result['response'],
    context=new_context1
)

# Define GEval metric for relevance
geval_metric = GEval(
    name="Relevance Evaluation",
    criteria="The response should be relevant to the input question and information provided in the context.",
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.CONTEXT
    ],
    threshold=0.6,
    model="gpt-4o-mini"
)

# Run the evaluation
result = evaluate([test_case], [geval_metric])

# Print results
print(result)

✨ You're running DeepEval's latest Relevance Evaluation [GEval] Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ✅ Relevance Evaluation [GEval] (score: 0.9851952800088839, threshold: 0.6, strict: False, evaluation model: gpt-4o-mini, reason: The response directly addresses the question about Artificial Intelligence by providing a clear definition and relevant examples, such as virtual assistants and robotics. It effectively utilizes the context information, particularly the definition provided, and elaborates on the applications and advancements in AI, demonstrating a strong alignment with the evaluation steps., error: None)

For test case:

  - input: What is Artificial Intelligence?
  - actual output: Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.
  - expected output: None
  - context: ['NVIDIA makes chips for AI', 'AI is known as Artificial Intellige

⚠ WARNING: No hyperparameters logged.
» ]8;id=358876;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 5.5s | token cost: 0.00018314999999999998 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Relevance Evaluation [GEval]', threshold=0.6, success=True, score=0.9851952800088839, reason='The response directly addresses the question about Artificial Intelligence by providing a clear definition and relevant examples, such as virtual assistants and robotics. It effectively utilizes the context information, particularly the definition provided, and elaborates on the applications and advancements in AI, demonstrating a strong alignment with the evaluation steps.', strict_mode=False, evaluation_model='gpt-4o-mini', error=None, evaluation_cost=0.00018314999999999998, verbose_logs='Criteria:\nThe response should be relevant to the input question and information provided in the context. \n \nEvaluation Steps:\n[\n    "Identify the main question or request in the input to establish the primary focus.",\n    "Review the context provided to extract relevant information that supports or relates to the